In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import time

# 进销口分析

In [3]:
input1 = pd.read_excel('附件2：302家无信贷记录企业的相关数据.xlsx',sheet_name='进项发票信息')
output1 = pd.read_excel('附件2：302家无信贷记录企业的相关数据.xlsx',sheet_name='销项发票信息')

In [25]:
for num in range(124,426):
    id1 = 'E'+str(num)
    input1[input1['企业代号']==id1]

In [35]:
r_in=[]

for num in range(124,426):
    id1 = 'E'+str(num)
    tem = input1[input1['企业代号']==id1]
    all_num = tem.shape[0]
    fei_num = tem[tem['发票状态']=='作废发票'].shape[0]
    # if all_num==0:
    #     continue
    ratio = fei_num/all_num
    
    r_in.append(ratio)
    
r_out=[]

for num in range(124,426):
    id1 = 'E'+str(num)
    tem = output1[output1['企业代号']==id1]
    all_num = tem.shape[0]
    fei_num = tem[tem['发票状态']=='作废发票'].shape[0]
    # if all_num==0:
    #     continue
    ratio = fei_num/all_num
    r_out.append(ratio)

    

name=[]    
for num in range(124,426):
    id1 = 'E'+str(num)
    name.append(id1)
    

In [36]:
in_num=[]
for num in range(124,426):
    id1 = 'E'+str(num)
    tem = input1[input1['企业代号']==id1]
    all_num = tem.shape[0]
    
    in_num.append(all_num)
    
out_num=[]
for num in range(124,426):
    id1 = 'E'+str(num)
    tem = output1[output1['企业代号']==id1]
    all_num = tem.shape[0]
    
    out_num.append(all_num)

In [39]:
data = {'企业名称':name,
    '进项发票的作废比例':r_in   ,
      '销项发票的作废比例':r_out  ,
        '进项发票数':in_num ,
        '销项发票数':out_num
}
df=pd.DataFrame(data=data)
# df = pd.DataFrame.from_dict(data,orient='index')

In [40]:
df.to_csv('附件2作废发票比例.csv',encoding='utf-8')

# 企业文字提取

In [59]:
data = pd.read_excel('附件1.xlsx',sheet_name='企业信息')
data

,企业代号,企业名称,信誉评级,是否违约
0,E1,***电器销售有限公司,A,否
1,E2,***技术有限责任公司,A,否
2,E3,***电子(中国)有限公司***分公司,C,否
3,E4,***发展有限责任公司,C,否
4,E5,***供应链管理有限公司,B,否
...,...,...,...,...
118,E119,***药房,D,是
119,E120,***陈列广告有限公司,D,是
120,E121,***药业连锁有限公司***药店,D,是
121,E122,***商贸有限责任公司,D,是


# 时间序列分析

In [41]:
for i in range(len(input1)):         # 删除作废发票
    if input1.loc[i,'发票状态']=='作废发票':
        input1.drop(i,inplace=True)

In [42]:
for i in range(len(output1)):
    if output1.loc[i,'发票状态']=='作废发票':
        output1.drop(i,inplace=True)

In [43]:
input1.reset_index(drop = True,inplace=True)
output1.reset_index(drop = True,inplace=True)

In [44]:
input1['开票日期'] = pd.to_datetime(input1['开票日期'])
output1['开票日期'] = pd.to_datetime(output1['开票日期'])

In [46]:
deal1=[]
deal2=[]
deal3=[]
deal4=[]
deal5=[]
for num in range(124,426):      # 遍历目标企业
    l_1 = 0
    l_2 = 0
    l_3 = 0       # 一年、两年、三年、四年的交易方
    l_4 = 0
    l_5 = 0

    id1 = 'E'+str(num)
    a = input1[input1['企业代号']==id1]
    a = a.reset_index(drop=True)
    other = np.unique(a['销方单位代号'])    # 交易方代号的唯一值
    
    year = a['开票日期'][len(a)-1].year - a['开票日期'][0].year+1  
    begin = a['开票日期'][0].year
    
    for i in range(len(other)):       # 遍历交易企业
        cou=0
        company = other[i]      # 选中的交易企业
        for j in range(year):    # 遍历每一年
            tem = a[(a['开票日期'] >str(begin+j) )& (a['开票日期'] <str(begin+j+1))]['销方单位代号'].unique()  # 一年内的交易企业
            if company in tem:
                cou +=1
                
        if cou == 1:
            l_1+=1
        elif cou == 2:
            l_2+=1
        elif cou ==3:
            l_3+= 1
        elif cou==4:
            l_4+=1
        elif cou == 5:
            l_5+=1
                
                
    deal1.append(l_1)
    deal2.append(l_2)
    deal3.append(l_3)        
    deal4.append(l_4)        
    deal5.append(l_5)

In [47]:
da1 = {'持续一年期的交易企业个数':deal1,
     '持续二年期的交易企业个数': deal2,
     '持续三年期的交易企业个数': deal3,
     '持续四年前的交易企业个数':deal4,
       '持续五年期的交易企业个数':deal5
     }

In [48]:
jx = pd.DataFrame(data=da1)


In [49]:
jx.to_csv('上游持续交易企业数.csv',encoding='utf-8')

In [50]:
deal1=[]
deal2=[]
deal3=[]
deal4=[]
for num in range(124,426):      # 遍历目标企业
    l_1 = 0
    l_2 = 0
    l_3 = 0       # 一年、两年、三年、四年的交易方
    l_4 = 0

    id1 = 'E'+str(num)
    a = output1[output1['企业代号']==id1]
    a = a.reset_index(drop=True)
    other = np.unique(a['购方单位代号'])    # 交易方代号的唯一值
    
    year = a['开票日期'][len(a)-1].year - a['开票日期'][0].year+1  
    begin = a['开票日期'][0].year
    
    for i in range(len(other)):       # 遍历交易企业
        cou=0
        company = other[i]      # 选中的交易企业
        for j in range(year):    # 遍历每一年
            tem = a[(a['开票日期'] >str(begin+j) )& (a['开票日期'] <str(begin+j+1))]['购方单位代号'].unique()  # 一年内的交易企业
            if company in tem:
                cou +=1
                
        if cou == 1:
            l_1+=1
        elif cou == 2:
            l_2+=1
        elif cou ==3:
            l_3+= 1
        elif cou==4:
            l_4+=1

                
                
    deal1.append(l_1)
    deal2.append(l_2)
    deal3.append(l_3)        
    deal4.append(l_4)        


In [51]:
da2= {'持续一年期的交易企业个数':deal1,
     '持续二年期的交易企业个数': deal2,
     '持续三年期的交易企业个数': deal3,
     '持续四年前的交易企业个数':deal4
     }

xx = pd.DataFrame(data=da2)


In [53]:
xx.to_csv('下游持续交易企业.csv',encoding='utf-8')

In [54]:
in_y=[]
for num in range(124,426):      # 遍历目标企业
    l_1 = 0
    l_2 = 0
    l_3 = 0       # 一年、两年、三年、四年的交易方
    l_4 = 0
    l_5 = 0

    id1 = 'E'+str(num)
    a = input1[input1['企业代号']==id1]
    a = a.reset_index(drop=True)
    other = np.unique(a['销方单位代号'])    # 交易方代号的唯一值
    
    year = a['开票日期'][len(a)-1].year - a['开票日期'][0].year+1  
    in_y.append(year)



out_y=[]
for num in range(124,426):      # 遍历目标企业
    l_1 = 0
    l_2 = 0
    l_3 = 0       # 一年、两年、三年、四年的交易方
    l_4 = 0

    id1 = 'E'+str(num)
    a = output1[output1['企业代号']==id1]
    a = a.reset_index(drop=True)
    other = np.unique(a['购方单位代号'])    # 交易方代号的唯一值
    
    year = a['开票日期'][len(a)-1].year - a['开票日期'][0].year+1  
    out_y.append(year)

In [55]:
new_in = input1[(input1['开票日期']<'2020')]    # 2020年之前的进项数据
new_out = output1[(output1['开票日期']<'2020')]

In [57]:
new_in

,企业代号,发票号码,开票日期,销方单位代号,金额,税额,价税合计,发票状态
0,E124,18891676,2017-09-01 00:00:00,C00014,338.46,57.54,396.00,有效发票
1,E124,18691267,2017-09-01 00:00:00,C00480,230.10,6.90,237.00,有效发票
2,E124,12995412,2017-09-01 00:00:00,C23675,223.30,6.70,230.00,有效发票
3,E124,6378193,2017-09-01 11:11:03,C00333,90090.09,9909.91,100000.00,有效发票
4,E124,6378194,2017-09-01 11:12:01,C00333,90090.09,9909.91,100000.00,有效发票
...,...,...,...,...,...,...,...,...
377930,E425,47732334,2019-11-13 00:00:00,C15032,320.51,21.11,341.62,有效发票
377931,E425,73996051,2019-11-20 00:00:00,C02766,2000.00,0.00,2000.00,有效发票
377932,E425,25962148,2019-12-09 00:00:00,C01937,138.03,16.00,154.03,有效发票
377933,E425,10726350,2019-12-23 00:00:00,C05583,264.60,34.40,299.00,有效发票


In [58]:
ur = []

dd = []
for num in range(124,426):      # 遍历目标企业
    
    id1 = 'E'+str(num)
    a = new_in[new_in['企业代号']==id1]
    a = a.reset_index(drop=True)
    
    end_1 = a['开票日期'][len(a)-1].year
    begin_1 = a['开票日期'][0].year
    
    begin_ji_1 = a[(a['开票日期']>str(begin_1))&(a['开票日期']<str(begin_1+1))]['金额'].sum()
    end_ji_1 =  a[(a['开票日期']>str(end_1))&(a['开票日期']<str(end_1+1))]['金额'].sum()
    
    
    b = new_out[new_out['企业代号']==id1]
    b = b.reset_index(drop=True)
    
    end_2 = b['开票日期'][len(b)-1].year
    begin_2 = b['开票日期'][0].year
    
    begin_ji_2 = b[(b['开票日期']>str(begin_2))&(b['开票日期']<str(begin_2+1))]['金额'].sum()
    end_ji_2 =  b[(b['开票日期']>str(end_2))&(b['开票日期']<str(end_2+1))]['金额'].sum()
    
    up_rate = (end_ji_2-end_ji_1)/(begin_ji_2-begin_ji_1)
    
    d = (end_ji_2-end_ji_1)-(begin_ji_2-begin_ji_1)
    
    
    ur.append(up_rate)
    dd.append(d)

In [59]:
dd

[-45789845.74000004,
 -82921856.73999998,
 117045572.13,
 -82361225.38999999,
 80992507.48999998,
 -17241774.779999986,
 35203073.05999999,
 8017782.599999994,
 9318509.319999985,
 32831925.79000002,
 55061109.830000006,
 34560158.61,
 1939896.8700000048,
 21863305.859999944,
 25765508.65,
 -14444900.569999993,
 513817.1300000176,
 18367475.44999999,
 16407932.780000003,
 36976839.98,
 61674934.760000005,
 5561132.810000002,
 -6770884.439999995,
 -460704.60000000196,
 21206898.52000001,
 32489189.65,
 4724051.869999992,
 9595999.599999998,
 45005966.83,
 -99670948.42999998,
 2442341.9899999984,
 3159678.0000000056,
 -31905244.53000001,
 -2977014.199999989,
 7341437.959999995,
 -10918369.670000002,
 -9261711.029999994,
 40525828.199999996,
 34711118.2,
 -1877310.2099999972,
 -3267715.999999989,
 13937874.020000003,
 40586869.25000001,
 4910909.329999998,
 26083586.749999996,
 -2219400.5699999947,
 14914113.40000001,
 7005452.539999995,
 13627940.259999994,
 -3394198.1299999915,
 -103660

In [60]:
data3 = {'绝对数变化':dd,
        '比例变化':ur}
df3 = pd.DataFrame(data=data3)
df3

,绝对数变化,比例变化
0,-4.578985e+07,-0.902160
1,-8.292186e+07,-0.278589
2,1.170456e+08,2.482219
3,-8.236123e+07,0.662969
4,8.099251e+07,2.935291
...,...,...
297,-5.554860e+04,0.110407
298,1.956756e+04,8.608094
299,5.829610e+03,0.599515
300,-8.231466e+04,2.014188


In [61]:
for i in range(len(df3)):
    if (df3.loc[i,'绝对数变化']>0) & (df3.loc[i,'比例变化']<0):     # 说明扭亏为盈
        df3.loc[i,'是否扭亏为盈利']=1
    else:
        df3.loc[i,'是否扭亏为盈利']=0

for i in range(len(df3)):
    if (df3.loc[i,'绝对数变化']<0) & (df3.loc[i,'比例变化']<0):     # 说明扭亏为盈
        df3.loc[i,'是否变为亏损']=1
    else:
        df3.loc[i,'是否变为亏损']=0        

df3

,绝对数变化,比例变化,是否扭亏为盈利,是否变为亏损
0,-4.578985e+07,-0.902160,0.0,1.0
1,-8.292186e+07,-0.278589,0.0,1.0
2,1.170456e+08,2.482219,0.0,0.0
3,-8.236123e+07,0.662969,0.0,0.0
4,8.099251e+07,2.935291,0.0,0.0
...,...,...,...,...
297,-5.554860e+04,0.110407,0.0,0.0
298,1.956756e+04,8.608094,0.0,0.0
299,5.829610e+03,0.599515,0.0,0.0
300,-8.231466e+04,2.014188,0.0,0.0


In [62]:
df3.to_csv('发展程度.csv',encoding='utf-8')

In [290]:
num=5
id1 = 'E'+str(num)
a = new_in[new_in['企业代号']==id1]
a = a.reset_index(drop=True)
    
end_1 = a['开票日期'][len(a)-1].year
begin_1 = a['开票日期'][0].year
    
begin_ji_1 = a[(a['开票日期']>str(begin_1))&(a['开票日期']<str(begin_1+1))]['金额'].sum()
end_ji_1 =  a[(a['开票日期']>str(end_1))&(a['开票日期']<str(end_1+1))]['金额'].sum()
    
    
b = new_out[new_out['企业代号']==id1]
b = b.reset_index(drop=True)
    
end_2 = b['开票日期'][len(b)-1].year
begin_2 = b['开票日期'][0].year
    
begin_ji_2 = b[(b['开票日期']>str(begin_2))&(b['开票日期']<str(begin_2+1))]['金额'].sum()
end_ji_2 =  b[(b['开票日期']>str(end_2))&(b['开票日期']<str(end_2+1))]['金额'].sum()

up_rate = (end_ji_2-end_ji_1)/(begin_ji_2-begin_ji_1)

In [291]:
print(end_ji_2-end_ji_1)
print(begin_ji_2-begin_ji_1)

4116517.7299999893
48300.580000001006


In [289]:
if 

[1.7206138607491175,
 2.1976741755350835,
 0.5163533480678486,
 0.053642120805519077,
 85.22708692110744,
 6.220033757117115,
 0.28299993853695593,
 1.1903434951584118,
 1.1026416281740918,
 5.6952388287385975,
 0.06224696926670617,
 0.3142310613924136,
 1.6246590416677933,
 1.2608609677500382,
 2.030952353418196,
 4.42752647859875,
 4.6091111208456725,
 25.37000437751128,
 1.2952080964666253,
 3.448482477010922,
 -107.85542263470977,
 -68.36411555912315,
 -3.1526842431065614,
 3.7508149740262886,
 1.8719903885132894,
 -1.5351713542943943,
 -4.576462137815929,
 7.404078615947699,
 1.8563019203053552,
 1.2348027347399602,
 1.6791198336803776,
 2.0636008405295763,
 0.5125450991083275,
 140.72763660986513,
 -11.283839443261392,
 3.21411258457861,
 0.9453293654883062,
 1.3313401805724063,
 7.452725580324382,
 6.115626835894702,
 4.989140651177328,
 12.135349485779685,
 1.1820756853858494,
 6.562504223791024,
 -5.818097757171771,
 12.069900028107975,
 2.966145392466539,
 5.250937640696781,


# 第二问发票数据处理

In [63]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import time

input1 = pd.read_excel('附件2：302家无信贷记录企业的相关数据.xlsx',sheet_name='进项发票信息')
output1 = pd.read_excel('附件2：302家无信贷记录企业的相关数据.xlsx',sheet_name='销项发票信息')

In [64]:
# r_in=[]

# for num in range(1,303):
#     id1 = 'E'+str(num+123)
#     tem = input1[input1['企业代号']==id1]
#     all_num = tem.shape[0]
#     fei_num = tem[tem['发票状态']=='作废发票'].shape[0]
#     ratio = fei_num/all_num
    
#     r_in.append(ratio)
    
r_out=[]

for num in range(1,303):
    id1 = 'E'+str(num+123)
    tem = output1[output1['企业代号']==id1]
    all_num = tem.shape[0]
    fei_num = tem[tem['发票状态']=='作废发票'].shape[0]
    ratio = fei_num/all_num
    
    r_out.append(ratio)

    

name=[]    
for num in range(1,303):
    id1 = 'E'+str(num+123)
    name.append(id1)
    

In [65]:
in_num=[]
for num in range(1,303):
    id1 = 'E'+str(num+123)
    tem = input1[input1['企业代号']==id1]
    all_num = tem.shape[0]
    
    in_num.append(all_num)
    
out_num=[]
for num in range(1,303):
    id1 = 'E'+str(num+123)
    tem = output1[output1['企业代号']==id1]
    all_num = tem.shape[0]
    
    out_num.append(all_num)

In [66]:
data = {'企业名称':name,
    '进项发票的作废比例':r_in   ,
      '销项发票的作废比例':r_out  
}

df=pd.DataFrame(data=data)
df

,企业名称,进项发票的作废比例,销项发票的作废比例
0,E124,0.123313,0.150039
1,E125,0.123817,0.134796
2,E126,0.033771,0.166227
3,E127,0.017931,0.030303
4,E128,0.027312,0.074900
...,...,...,...
297,E421,0.000000,0.034483
298,E422,0.000000,0.100000
299,E423,0.000000,0.142857
300,E424,0.000000,0.139535


In [11]:
df.to_csv('第二问作废比例.csv',encoding='gbk')

In [12]:
new_in = input1[(input1['开票日期']<'2020')]    # 2020年之前的进项数据
new_out = output1[(output1['开票日期']<'2020')]

In [67]:
ur = []

dd = []
for num in range(1,303):      # 遍历目标企业
    
    id1 = 'E'+str(num+123)
    a = new_in[new_in['企业代号']==id1]
    a = a.reset_index(drop=True)
    
    end_1 = a['开票日期'][len(a)-1].year
    begin_1 = a['开票日期'][0].year
    
    begin_ji_1 = a[(a['开票日期']>str(begin_1))&(a['开票日期']<str(begin_1+1))]['金额'].sum()
    end_ji_1 =  a[(a['开票日期']>str(end_1))&(a['开票日期']<str(end_1+1))]['金额'].sum()
    
    
    b = new_out[new_out['企业代号']==id1]
    b = b.reset_index(drop=True)
    
    end_2 = b['开票日期'][len(b)-1].year
    begin_2 = b['开票日期'][0].year
    
    begin_ji_2 = b[(b['开票日期']>str(begin_2))&(b['开票日期']<str(begin_2+1))]['金额'].sum()
    end_ji_2 =  b[(b['开票日期']>str(end_2))&(b['开票日期']<str(end_2+1))]['金额'].sum()
    
    up_rate = (end_ji_2-end_ji_1)/(begin_ji_2-begin_ji_1)
    
    d = (end_ji_2-end_ji_1)-(begin_ji_2-begin_ji_1)
    
    
    ur.append(up_rate)
    dd.append(d)

In [68]:
data3 = {'绝对数变化':dd,
        '比例变化':ur}
df3 = pd.DataFrame(data=data3)
df3

,绝对数变化,比例变化
0,-4.578985e+07,-0.902160
1,-8.292186e+07,-0.278589
2,1.170456e+08,2.482219
3,-8.236123e+07,0.662969
4,8.099251e+07,2.935291
...,...,...
297,-5.554860e+04,0.110407
298,1.956756e+04,8.608094
299,5.829610e+03,0.599515
300,-8.231466e+04,2.014188


In [69]:
for i in range(len(df3)):
    if (df3.loc[i,'绝对数变化']>0) & (df3.loc[i,'比例变化']<0):     # 说明扭亏为盈
        df3.loc[i,'是否扭亏为盈利']=1
    else:
        df3.loc[i,'是否扭亏为盈利']=0

for i in range(len(df3)):
    if (df3.loc[i,'绝对数变化']<0) & (df3.loc[i,'比例变化']<0):     # 说明扭亏为盈
        df3.loc[i,'是否变为亏损']=1
    else:
        df3.loc[i,'是否变为亏损']=0        

df3

,绝对数变化,比例变化,是否扭亏为盈利,是否变为亏损
0,-4.578985e+07,-0.902160,0.0,1.0
1,-8.292186e+07,-0.278589,0.0,1.0
2,1.170456e+08,2.482219,0.0,0.0
3,-8.236123e+07,0.662969,0.0,0.0
4,8.099251e+07,2.935291,0.0,0.0
...,...,...,...,...
297,-5.554860e+04,0.110407,0.0,0.0
298,1.956756e+04,8.608094,0.0,0.0
299,5.829610e+03,0.599515,0.0,0.0
300,-8.231466e+04,2.014188,0.0,0.0


In [71]:
df3.to_csv('第二问绝对数和比例变化.csv',encoding='utf-8')